# Sparse Tensors

```
Copyright 2022 National Technology & Engineering Solutions of Sandia,
LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS, the
U.S. Government retains certain rights in this software.
```

The Sptensor class stores the data in coordinate format.

## Contents

* [Creating a Sptensor](#Creating-a-Sptensor)
* [Creating a one-dimensional Sptensor](#Creating-a-one-dimensional-Sptensor)
* [Creating an all-zero Sptensor](#Creating-an-all-zero-Sptensor)
* [Creating an empty Sptensor](#Creating-an-empty-Sptensor)
* [Convert a sparse tensor into a dense tensor](#Convert-a-sparse-tensor-into-a-dense-tensor)
* [Use ndims and size to get the size of a Sptensor](#Use-ndims-and-size-to-get-the-size-of-a-Sptensor)
* [Use nnz to get the number of nonzeroes of a Sptensor](#Use-nnz-to-get-the-number-of-nonzeroes-of-a-Sptensor)
* [Nonzero reference for a Sptensor](#Nonzero-reference-for-a-Sptensor)
* [Nonzero assignment for a Sptensor](#Nonzero-assignment-for-a-Sptensor)
* [Subscripted reference for a Sptensor](#Subscripted-reference-for-a-Sptensor)
* [Subscripted assignment for a Sptensor](#Subscripted-assignment-for-a-Sptensor)
* [Basic operations on a Sptensor](#Basic-operations-on-a-Sptensor)
     * [Addition](#Addition)
     * [Subtraction](#Subtraction)
     * [Multiplication](#Multiplication)
     * [Division](#Division)
     * [Power](#Power)
     * [Equality](#Equality)
* [Displaying a Sptensor](#Displaying-a-Sptensor)

## Creating a Sptensor

A sparse tensor can be created by passing in a list of subscripts and values. For example, here we pass in three subscripts and a scalar value. The resuling sparse tensor has three nonzero entries, and the shape is the size of the largest subscript in each dimension.

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath('../'))
import pyttb as ttb
import numpy as np

subs =  np.array([[1,1,1],[1,2,1],[3,4,2]])
vals = np.array([[1],[2],[3]]) # Vals is a column vector
X = ttb.sptensor.from_aggregator(subs,vals)
print(X)

In [ ]:
X = ttb.sptensor.from_data(subs,vals,(4,6,3))   #Or, specify the shape explicitly
print(X)

Values corresponding to repeated subscripts are summed.

In [ ]:
subs = np.array([[1,1,1],[1,1,3],[2,2,2],[4,4,4],[1,1,1],[1,1,1]])  # (1,1,1) is repeated
vals = np.array([2,2,2,2,2,2])[:, None]  # Vals is a column vector
X = ttb.sptensor.from_aggregator(subs,vals)
print(X)

## Creating a one-dimensional Sptensor

In [ ]:
X = ttb.sptensor.from_aggregator(np.array([[1],[2],[3]]),np.array([[2],[2],[2]]))
print(X)

## Creating an all-zero Sptensor

In [ ]:
X = ttb.sptensor()
X[10,10,10] = 0
print(X)

## Creating an empty Sptensor

In [ ]:
X = ttb.sptensor()
print(X)

## Convert a sparse tensor into a dense tensor

In [ ]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[1],[1]]), (2,2))
X = ttb.tensor.from_tensor_type(X)
print(X)

## Use ndims and size to get the size of a Sptensor

In [ ]:
X = ttb.sptensor.from_data(np.array([[1,1,1],[2,3,2],[3,4,1],[1,0,0]]),np.array([[3],[2],[1],[3]]), (4, 5, 3))
print(X)

In [ ]:
X.ndims

In [ ]:
X.shape

## Use nnz to get the number of nonzeroes of a Sptensor

In [ ]:
X.nnz

## Nonzero reference for a Sptensor

In [ ]:
X[1, 1, 1]

## Nonzero assignment for a Sptensor

In [ ]:
X[2, 2, 2] = 3.3
print(X)

You can also modify just the value

In [ ]:
X[2, 2, 2] = -44.1
print(X)

## Subscripted reference for a Sptensor

In [ ]:
print(X)

In [ ]:
X[1,0,0]

In [ ]:
X[1,2,1]   #Not a nonzero

## Subscripted assignment for a Sptensor

In [ ]:
X[1,2,1] = 5
print(X[1,2,1])

In [ ]:
print(X)   #Note that (1,2,1) is now a nonzero

## Basic operations on a Sptensor

Sptensors support plus, minus, times, divide, power, equals, and not-equals operators.  Sptensors can use there operators with another Sptensor or a scalar (with the exception of equalities which only takes Sptensors).  All mathematical operators are elementwise operations.

### Addition

In [ ]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[2],[2]]),(2,2))
print(X)

In [ ]:
Y = ttb.sptensor.from_data(np.array([[0,0],[0,1]]),np.array([[3],[3]]),(2,2))
print(Y)

In [ ]:
X + 1  # This addition yields dense tensor

In [ ]:
X + Y  # This addition yields sparse tensor

In [ ]:
X += 2
print(X)

### Subtraction

In [ ]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[2],[2]]),(2,2))
print(X)

In [ ]:
Y = ttb.sptensor.from_data(np.array([[0,0],[0,1]]),np.array([[3],[3]]),(2,2))
print(Y)

In [ ]:
X - 1

In [ ]:
X - Y

In [ ]:
X -= 2
print(X)

### Multiplication

In [ ]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[2],[2]]),(2,2))
print(X)

In [ ]:
Y = ttb.sptensor.from_data(np.array([[0,0],[0,1]]),np.array([[3],[3]]),(2,2))
print(Y)

In [ ]:
X * 2.5

In [ ]:
X * Y

In [ ]:
X *= 3.3
print(X)

### Division

In [ ]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[2],[2]]),(2,2))
print(X)

In [ ]:
Y = ttb.sptensor.from_data(np.array([[0,0],[0,1]]),np.array([[3],[3]]),(2,2))
print(Y)

In [ ]:
X / 3

In [ ]:
X / Y

In [ ]:
X /= 4
print(X)

### Equality

In [ ]:
X = ttb.sptensor.from_data(np.array([[0,0],[1,1]]),np.array([[2],[2]]),(2,2))
print(X)

In [ ]:
Y = ttb.sptensor.from_data(np.array([[0,0],[0,1]]),np.array([[3],[3]]),(2,2))
print(Y)

In [ ]:
(X == Y)

In [ ]:
X.isequal(Y)

In [ ]:
X != Y

## Displaying a Sptensor

In [ ]:
print(X)

In [ ]:
X     #In the python interface